In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define the image size and the number of classes
IMG_SIZE = 224
NUM_CLASSES = 4

# Load the images and labels
images = []
labels = []
for i, error_type in enumerate(['Functional', 'System_level', 'Usability_level', 'Security level']):
    folder_path = f'/content/sample_data/Dataset/{error_type}'
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(i)

# Convert to numpy arrays and normalize the images
images = np.array(images) / 255.0
labels = np.array(labels)

# Split the data into training and test sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
import tensorflow as tf

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES)
])

# Compile the model
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))


Epoch 1/10
4/4 [==============================] - 11s 139ms/step - loss: 27.4158 - accuracy: 0.2109 - val_loss: 5.4178 - val_accuracy: 0.2121
Epoch 2/10
4/4 [==============================] - 0s 26ms/step - loss: 3.2984 - accuracy: 0.2578 - val_loss: 2.1562 - val_accuracy: 0.2727
Epoch 3/10
4/4 [==============================] - 0s 25ms/step - loss: 1.6914 - accuracy: 0.3359 - val_loss: 1.4308 - val_accuracy: 0.2727
Epoch 4/10
4/4 [==============================] - 0s 26ms/step - loss: 1.3649 - accuracy: 0.3047 - val_loss: 1.4430 - val_accuracy: 0.2121
Epoch 5/10
4/4 [==============================] - 0s 27ms/step - loss: 1.1935 - accuracy: 0.4609 - val_loss: 1.4732 - val_accuracy: 0.2727
Epoch 6/10
4/4 [==============================] - 0s 25ms/step - loss: 1.0036 - accuracy: 0.5938 - val_loss: 1.3388 - val_accuracy: 0.3030
Epoch 7/10
4/4 [==============================] - 0s 25ms/step - loss: 0.8688 - accuracy: 0.6406 - val_loss: 1.5484 - val_accuracy: 0.3636
Epoch 8/10
4/4 [========

In [ ]:
model.save('bug_classifier.h5')


In [4]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.6 MB/s eta 0:00:00


In [ ]:

import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# Let's assume you have data and labels
# data is a list of texts extracted from the prescriptions
# labels is a list of the exact prescriptions
data = [f"text{i}" for i in range(1, 101)]
labels = [f"prescription{i}" for i in range(1, 101)]


# Initialize the LabelEncoder
le = LabelEncoder()

# Fit the LabelEncoder on all labels
le.fit(labels)

# Transform the labels
labels = le.transform(labels)

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Tokenize the data
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=128)

# Convert encodings to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

# Compile the model
model.compile(optimizer=Adam(learning_rate=5e-5),
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=[SparseCategoricalAccuracy()])

# Train the model
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16)

# Now you can use the trained model to predict the exact prescriptions on new, unseen data



All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
5/5 [==============================] - 52s 185ms/step - loss: nan - sparse_categorical_accuracy: 0.0000e+00
Epoch 2/3
5/5 [==============================] - 1s 127ms/step - loss: nan - sparse_categorical_accuracy: 0.0000e+00
Epoch 3/3
5/5 [==============================] - 1s 125ms/step - loss: nan - sparse_categorical_accuracy: 0.0000e+00


In [6]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the image size and the number of classes
IMG_SIZE = 224
NUM_CLASSES = 4

# Load the images and labels
images = []
labels = []
for i, error_type in enumerate(['Functional', 'System_level', 'Usability_level', 'Security level']):
    folder_path = f'/content/sample_data/Dataset/{error_type}'
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(i)

# Convert to numpy arrays and normalize the images
images = np.array(images) / 255.0
labels = np.array(labels)

# Split the data into training and test sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

import tensorflow as tf

# Load pre-trained VGG16 model without the top layers
base_model = VGG16(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False

# Define the model
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    Dropout(0.5),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# Train the model
batch_size = 32
history = model.fit(datagen.flow(train_images, train_labels, batch_size=batch_size),
                    steps_per_epoch=len(train_images) // batch_size,
                    validation_data=(test_images, test_labels),
                    epochs=30)

model.save('bug_classifier.h5')


Epoch 1/30
4/4 [==============================] - 4s 401ms/step - loss: 1.5646 - accuracy: 0.2031 - val_loss: 1.3761 - val_accuracy: 0.2121
Epoch 2/30
4/4 [==============================] - 2s 419ms/step - loss: 1.4319 - accuracy: 0.2422 - val_loss: 1.3940 - val_accuracy: 0.2727
Epoch 3/30
4/4 [==============================] - 2s 403ms/step - loss: 1.4684 - accuracy: 0.2500 - val_loss: 1.3638 - val_accuracy: 0.3939
Epoch 4/30
4/4 [==============================] - 2s 402ms/step - loss: 1.3939 - accuracy: 0.2500 - val_loss: 1.3701 - val_accuracy: 0.3030
Epoch 5/30
4/4 [==============================] - 2s 471ms/step - loss: 1.3722 - accuracy: 0.3203 - val_loss: 1.3602 - val_accuracy: 0.3636
Epoch 6/30
4/4 [==============================] - 2s 426ms/step - loss: 1.3641 - accuracy: 0.3281 - val_loss: 1.3511 - val_accuracy: 0.4545
Epoch 7/30
4/4 [==============================] - 2s 373ms/step - loss: 1.3471 - accuracy: 0.3125 - val_loss: 1.3444 - val_accuracy: 0.3636
Epoch 8/30
4/4 [====

In [7]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_images, test_labels, batch_size=128)
print("test loss, test accuracy:", results)


Evaluate on test data
1/1 [==============================] - 5s 5s/step - loss: 1.2763 - accuracy: 0.4545
test loss, test accuracy: [1.276265263557434, 0.4545454680919647]


In [9]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Define the image size and the number of classes
IMG_SIZE = 224
NUM_CLASSES = 4

# Load the images and labels
images = []
labels = []
for i, error_type in enumerate(['Functional', 'System_level', 'Usability_level', 'Security level']):
    folder_path = f'/content/sample_data/Dataset/{error_type}'
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(i)

# Convert to numpy arrays and normalize the images
images = np.array(images) / 255.0
labels = np.array(labels)

# Split the data into training and test sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

import tensorflow as tf

# Load pre-trained VGG16 model without the top layers
base_model = VGG16(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')

# Make last block of the base_model trainable
for layer in base_model.layers[-4:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

# Define the model
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    Dropout(0.5),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model
batch_size = 32
history = model.fit(datagen.flow(train_images, train_labels, batch_size=batch_size),
                    steps_per_epoch=len(train_images) // batch_size,
                    validation_data=(test_images, test_labels),
                    epochs=50,
                    callbacks=[early_stopping])

model.save('bug_classifier.h5')

Epoch 1/50
4/4 [==============================] - 9s 535ms/step - loss: 1.7848 - accuracy: 0.2500 - val_loss: 1.3999 - val_accuracy: 0.2121
Epoch 2/50
4/4 [==============================] - 2s 433ms/step - loss: 1.4305 - accuracy: 0.2188 - val_loss: 1.3869 - val_accuracy: 0.1818
Epoch 3/50
4/4 [==============================] - 2s 469ms/step - loss: 1.4042 - accuracy: 0.2188 - val_loss: 1.3875 - val_accuracy: 0.2121
Epoch 4/50
4/4 [==============================] - 2s 504ms/step - loss: 1.4017 - accuracy: 0.2734 - val_loss: 1.3892 - val_accuracy: 0.2121
Epoch 5/50
4/4 [==============================] - 2s 471ms/step - loss: 1.3847 - accuracy: 0.3047 - val_loss: 1.3944 - val_accuracy: 0.2121


In [10]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_images, test_labels, batch_size=128)
print("test loss, test accuracy:", results)


Evaluate on test data
1/1 [==============================] - 0s 237ms/step - loss: 1.3944 - accuracy: 0.2121
test loss, test accuracy: [1.3943809270858765, 0.21212121844291687]


In [12]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the image size and the number of classes
IMG_SIZE = 224
NUM_CLASSES = 4

# Load the images and labels
images = []
labels = []
for i, error_type in enumerate(['Functional', 'System_level', 'Usability_level', 'Security level']):
    folder_path = f'/content/sample_data/Dataset/{error_type}'
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(i)

# Convert to numpy arrays and normalize the images
images = np.array(images) / 255.0
labels = np.array(labels)

# Split the data into training and test sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

import tensorflow as tf

# Load pre-trained VGG16 model without the top layers
base_model = VGG16(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = True

# Define the model
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    Dropout(0.6),
    tf.keras.layers.Dense(128, activation='relu'),
    Dropout(0.6),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# Train the model
batch_size = 32
history = model.fit(datagen.flow(train_images, train_labels, batch_size=batch_size),
                    steps_per_epoch=len(train_images) // batch_size,
                    validation_data=(test_images, test_labels),
                    epochs=100)

model.save('bug_classifier.h5')


Epoch 1/100
4/4 [==============================] - 9s 548ms/step - loss: 1.5885 - accuracy: 0.2344 - val_loss: 1.3889 - val_accuracy: 0.1818
Epoch 2/100
4/4 [==============================] - 2s 478ms/step - loss: 1.4339 - accuracy: 0.1953 - val_loss: 1.3853 - val_accuracy: 0.3030
Epoch 3/100
4/4 [==============================] - 2s 427ms/step - loss: 1.4786 - accuracy: 0.2266 - val_loss: 1.3888 - val_accuracy: 0.3030
Epoch 4/100
4/4 [==============================] - 2s 426ms/step - loss: 1.3934 - accuracy: 0.2812 - val_loss: 1.3915 - val_accuracy: 0.3030
Epoch 5/100
4/4 [==============================] - 2s 424ms/step - loss: 1.4090 - accuracy: 0.2422 - val_loss: 1.3912 - val_accuracy: 0.2121
Epoch 6/100
4/4 [==============================] - 2s 427ms/step - loss: 1.4015 - accuracy: 0.2734 - val_loss: 1.3971 - val_accuracy: 0.2121
Epoch 7/100
4/4 [==============================] - 2s 427ms/step - loss: 1.3966 - accuracy: 0.2344 - val_loss: 1.3936 - val_accuracy: 0.2121
Epoch 8/100
4

In [13]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_images, test_labels, batch_size=128)
print("test loss, test accuracy:", results)


Evaluate on test data
1/1 [==============================] - 0s 296ms/step - loss: 1.3920 - accuracy: 0.2121
test loss, test accuracy: [1.3919548988342285, 0.21212121844291687]


In [15]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential

# Define the image size and the number of classes
IMG_SIZE = 224
NUM_CLASSES = 4

# Load the images and labels
images = []
labels = []
for i, error_type in enumerate(['Functional', 'System_level', 'Usability_level', 'Security level']):
    folder_path = f'/content/sample_data/Dataset/{error_type}'
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(i)

# Convert to numpy arrays and normalize the images
images = np.array(images) / 255.0
labels = np.array(labels)

# Split the data into training and test sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Load pre-trained ResNet50 model without the top layers
base_model = ResNet50(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = True

# Define the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# Train the model
batch_size = 32
history = model.fit(datagen.flow(train_images, train_labels, batch_size=batch_size),
                    steps_per_epoch=len(train_images) // batch_size,
                    validation_data=(test_images, test_labels),
                    epochs=200)

model.save('bug_classifier.h5')


Epoch 1/200
4/4 [==============================] - 40s 741ms/step - loss: 1.8533 - accuracy: 0.3203 - val_loss: 1.6207 - val_accuracy: 0.3030
Epoch 2/200
4/4 [==============================] - 2s 383ms/step - loss: 1.5218 - accuracy: 0.3906 - val_loss: 16.6120 - val_accuracy: 0.2121
Epoch 3/200
4/4 [==============================] - 2s 381ms/step - loss: 1.7956 - accuracy: 0.3281 - val_loss: 15740.3994 - val_accuracy: 0.2121
Epoch 4/200
4/4 [==============================] - 2s 372ms/step - loss: 1.5842 - accuracy: 0.3828 - val_loss: 52367.5664 - val_accuracy: 0.2121
Epoch 5/200
4/4 [==============================] - 2s 381ms/step - loss: 1.4770 - accuracy: 0.3906 - val_loss: 89108.1719 - val_accuracy: 0.2121
Epoch 6/200
4/4 [==============================] - 2s 435ms/step - loss: 1.8259 - accuracy: 0.3984 - val_loss: 517.8336 - val_accuracy: 0.2121
Epoch 7/200
4/4 [==============================] - 2s 494ms/step - loss: 1.8171 - accuracy: 0.3516 - val_loss: 23693932.0000 - val_accurac

In [17]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

# Define the image size and the number of classes
IMG_SIZE = 224
NUM_CLASSES = 4

# Load the images and labels
images = []
labels = []
for i, error_type in enumerate(['Functional', 'System_level', 'Usability_level', 'Security level']):
    folder_path = f'/content/sample_data/Dataset/{error_type}'
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(i)

# Convert to numpy arrays and normalize the images
images = np.array(images) / 255.0
labels = np.array(labels)

# Split the data into training and test sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Load pre-trained ResNet50 model without the top layers
base_model = ResNet50(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = True

# Define the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),  # Added L2 regularization
    Dropout(0.5),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),  # Added L2 regularization
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Train the model
batch_size = 32
history = model.fit(datagen.flow(train_images, train_labels, batch_size=batch_size),
                    steps_per_epoch=len(train_images) // batch_size,
                    validation_data=(test_images, test_labels),
                    epochs=200,
                    callbacks=[early_stopping])  # Added early stopping

model.save('bug_classifier.h5')


Epoch 1/200
4/4 [==============================] - 44s 742ms/step - loss: 8.1665 - accuracy: 0.2734 - val_loss: 8.4793 - val_accuracy: 0.2727
Epoch 2/200
4/4 [==============================] - 2s 386ms/step - loss: 7.7122 - accuracy: 0.3281 - val_loss: 8.0630 - val_accuracy: 0.3030
Epoch 3/200
4/4 [==============================] - 2s 364ms/step - loss: 7.2622 - accuracy: 0.3906 - val_loss: 12.5927 - val_accuracy: 0.2121
Epoch 4/200
4/4 [==============================] - 2s 379ms/step - loss: 7.3214 - accuracy: 0.3438 - val_loss: 11.1641 - val_accuracy: 0.2121
Epoch 5/200
4/4 [==============================] - 2s 393ms/step - loss: 6.8031 - accuracy: 0.3359 - val_loss: 11.8795 - val_accuracy: 0.2121
Epoch 6/200
4/4 [==============================] - 2s 468ms/step - loss: 6.7370 - accuracy: 0.3906 - val_loss: 15.3166 - val_accuracy: 0.2727
Epoch 7/200
4/4 [==============================] - 2s 369ms/step - loss: 6.2700 - accuracy: 0.3906 - val_loss: 29.9338 - val_accuracy: 0.2727
Epoch 8

In [18]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_images, test_labels, batch_size=128)
print("test loss, test accuracy:", results)


Evaluate on test data
1/1 [==============================] - 0s 168ms/step - loss: 65.3720 - accuracy: 0.2121
test loss, test accuracy: [65.37201690673828, 0.21212121844291687]


In [19]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Define the image size and the number of classes
IMG_SIZE = 224
NUM_CLASSES = 4

# Load the images and labels
images = []
labels = []
for i, error_type in enumerate(['Functional', 'System_level', 'Usability_level', 'Security level']):
    folder_path = f'/content/sample_data/Dataset/{error_type}'
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(i)

# Convert to numpy arrays and normalize the images
images = np.array(images) / 255.0
labels = np.array(labels)

# Split the data into training and test sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Load pre-trained ResNet50 model without the top layers
base_model = ResNet50(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = True

# Define the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),  # Reduced L2 regularization
    Dropout(0.5),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),  # Reduced L2 regularization
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model with a smaller learning rate
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Train the model
batch_size = 32
history = model.fit(datagen.flow(train_images, train_labels, batch_size=batch_size),
                    steps_per_epoch=len(train_images) // batch_size,
                    validation_data=(test_images, test_labels),
                    epochs=200,
                    callbacks=[early_stopping])  # Added early stopping

model.save('bug_classifier.h5')


Epoch 1/200
4/4 [==============================] - 41s 734ms/step - loss: 2.4452 - accuracy: 0.2734 - val_loss: 5.4125 - val_accuracy: 0.2121
Epoch 2/200
4/4 [==============================] - 2s 419ms/step - loss: 2.2187 - accuracy: 0.4375 - val_loss: 10.9659 - val_accuracy: 0.2121
Epoch 3/200
4/4 [==============================] - 2s 477ms/step - loss: 2.1792 - accuracy: 0.4141 - val_loss: 18.5271 - val_accuracy: 0.2121
Epoch 4/200
4/4 [==============================] - 2s 381ms/step - loss: 2.0954 - accuracy: 0.3672 - val_loss: 7.1400 - val_accuracy: 0.3030
Epoch 5/200
4/4 [==============================] - 2s 369ms/step - loss: 1.9100 - accuracy: 0.4141 - val_loss: 108.9987 - val_accuracy: 0.2121
Epoch 6/200
4/4 [==============================] - 2s 388ms/step - loss: 2.3826 - accuracy: 0.4375 - val_loss: 52.5191 - val_accuracy: 0.2121
Epoch 7/200
4/4 [==============================] - 2s 377ms/step - loss: 2.1880 - accuracy: 0.3828 - val_loss: 120.0152 - val_accuracy: 0.2727
Epoch

In [20]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_images, test_labels, batch_size=128)
print("test loss, test accuracy:", results)


Evaluate on test data
1/1 [==============================] - 0s 179ms/step - loss: 1646.4633 - accuracy: 0.2121
test loss, test accuracy: [1646.4632568359375, 0.21212121844291687]


In [23]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# Define the image size and the number of classes
IMG_SIZE = 224
NUM_CLASSES = 4

# Load the images and labels
images = []
labels = []
for i, error_type in enumerate(['Functional', 'System_level', 'Usability_level', 'Security level']):
    folder_path = f'/content/sample_data/Dataset/{error_type}'
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(i)

# Convert to numpy arrays and normalize the images
images = np.array(images) / 255.0
labels = np.array(labels)

# Split the data into training and test sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Load pre-trained ResNet50 model without the top layers
base_model = ResNet50(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Set this to false to use the original pre-trained weights

# Define the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model with a smaller learning rate
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# Train the model
batch_size = 32
history = model.fit(datagen.flow(train_images, train_labels, batch_size=batch_size),
                    steps_per_epoch=len(train_images) // batch_size,
                    validation_data=(test_images, test_labels),
                    epochs=50)  # Reduced to 50 epochs

model.save('bug_classifier.h5')


Epoch 1/50
4/4 [==============================] - 6s 635ms/step - loss: 1.7777 - accuracy: 0.2344 - val_loss: 1.4162 - val_accuracy: 0.2727
Epoch 2/50
4/4 [==============================] - 2s 405ms/step - loss: 1.6804 - accuracy: 0.2812 - val_loss: 1.4179 - val_accuracy: 0.3030
Epoch 3/50
4/4 [==============================] - 2s 396ms/step - loss: 1.6461 - accuracy: 0.2578 - val_loss: 1.4342 - val_accuracy: 0.2121
Epoch 4/50
4/4 [==============================] - 2s 370ms/step - loss: 1.5144 - accuracy: 0.2266 - val_loss: 1.3852 - val_accuracy: 0.3030
Epoch 5/50
4/4 [==============================] - 2s 362ms/step - loss: 1.5263 - accuracy: 0.1875 - val_loss: 1.3799 - val_accuracy: 0.3333
Epoch 6/50
4/4 [==============================] - 1s 350ms/step - loss: 1.4387 - accuracy: 0.2734 - val_loss: 1.3986 - val_accuracy: 0.2121
Epoch 7/50
4/4 [==============================] - 2s 364ms/step - loss: 1.3968 - accuracy: 0.3516 - val_loss: 1.3690 - val_accuracy: 0.2121
Epoch 8/50
4/4 [====

In [24]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_images, test_labels, batch_size=128)
print("test loss, test accuracy:", results)


Evaluate on test data
1/1 [==============================] - 0s 231ms/step - loss: 1.3453 - accuracy: 0.3939
test loss, test accuracy: [1.3452935218811035, 0.39393940567970276]


In [26]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# Define the image size and the number of classes
IMG_SIZE = 224
NUM_CLASSES = 4

# Load the images and labels
images = []
labels = []
for i, error_type in enumerate(['Functional', 'System_level', 'Usability_level', 'Security level']):
    folder_path = f'/content/sample_data/Dataset/{error_type}'
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(i)

# Convert to numpy arrays and normalize the images
images = np.array(images) / 255.0
labels = np.array(labels)

# Split the data into training and test sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Load pre-trained ResNet50 model without the top layers
base_model = ResNet50(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Set this to false to use the original pre-trained weights

# Define the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.7),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model with a smaller learning rate
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# Train the model
batch_size = 32
history = model.fit(datagen.flow(train_images, train_labels, batch_size=batch_size),
                    steps_per_epoch=len(train_images) // batch_size,
                    validation_data=(test_images, test_labels),
                    epochs=50)  # Reduced to 50 epochs

model.save('bug_classifier.h5')


Epoch 1/50
4/4 [==============================] - 7s 727ms/step - loss: 1.7415 - accuracy: 0.2109 - val_loss: 1.4265 - val_accuracy: 0.2121
Epoch 2/50
4/4 [==============================] - 2s 420ms/step - loss: 1.7677 - accuracy: 0.2109 - val_loss: 1.4073 - val_accuracy: 0.3030
Epoch 3/50
4/4 [==============================] - 2s 513ms/step - loss: 1.6614 - accuracy: 0.2500 - val_loss: 1.3878 - val_accuracy: 0.3030
Epoch 4/50
4/4 [==============================] - 2s 652ms/step - loss: 1.5187 - accuracy: 0.2188 - val_loss: 1.3850 - val_accuracy: 0.3030
Epoch 5/50
4/4 [==============================] - 2s 416ms/step - loss: 1.5812 - accuracy: 0.2422 - val_loss: 1.3825 - val_accuracy: 0.3030
Epoch 6/50
4/4 [==============================] - 2s 432ms/step - loss: 1.5492 - accuracy: 0.2500 - val_loss: 1.3846 - val_accuracy: 0.1818
Epoch 7/50
4/4 [==============================] - 2s 383ms/step - loss: 1.4409 - accuracy: 0.2422 - val_loss: 1.3846 - val_accuracy: 0.2424
Epoch 8/50
4/4 [====

In [27]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_images, test_labels, batch_size=128)
print("test loss, test accuracy:", results)


Evaluate on test data
1/1 [==============================] - 0s 174ms/step - loss: 1.3886 - accuracy: 0.2121
test loss, test accuracy: [1.3886091709136963, 0.21212121844291687]
